In [1]:
import pandas as pd
#import json

In [2]:
# прочитаем train_2_col.csv
train_4_col_2 = pd.read_csv('train_4_col_2.csv', sep='\t', index_col = 0)
train_4_col_2_small = train_4_col_2.head(100)
train_4_col_2_small.head()

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,id,category,subcategory
0,3edeb34cf93f490ff760af85,9,914
1,c98febd50dad3cc0ffc86085,22,2202
2,ade01e13912a46a99134cc75,22,2204
3,ab3e6941c11304c1519aef75,22,2209
5,252452a91c944a22c276d995,22,2204


In [3]:
# прочитаем train_4_col_3 .csv
train_4_col_3 = pd.read_csv('train_4_col_3.csv', sep='\t', index_col = 0)
train_4_col_3_small = train_4_col_3.head(100)
train_4_col_3_small.head()

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,id,fields
0,3edeb34cf93f490ff760af85,"[{'field': {'name': 'Женский гардероб', 'id': ..."
1,c98febd50dad3cc0ffc86085,"[{'field': {'name': 'Детский гардероб', 'id': ..."
2,ade01e13912a46a99134cc75,"[{'field': {'name': 'Детский гардероб', 'id': ..."
3,ab3e6941c11304c1519aef75,"[{'field': {'name': 'Детский гардероб', 'id': ..."
5,252452a91c944a22c276d995,"[{'field': {'name': 'Детский гардероб', 'id': ..."


In [4]:
# прочитаем train_4_col.csv
train_4_col = pd.read_csv('train_4_col.csv', sep='\t', index_col = 0)
train_4_col_small = train_4_col.head(100)
train_4_col_small.head()

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,id,name,description,price
0,3edeb34cf93f490ff760af85,Сумка DG,NaN,199900
1,c98febd50dad3cc0ffc86085,Комплект,8-12 лет,35000
2,ade01e13912a46a99134cc75,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",30000
3,ab3e6941c11304c1519aef75,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",80000
5,252452a91c944a22c276d995,Жилет теплый,NaN,50000


#### Объединим маленькие датафреймы

<!DOCTYPE html>
<html>
<head>
<style>
body {
    background-color: orange;
}    
</style> 
</head>
</html>


<font color="red" size="3">Внимание!!!.</font><br> 

чтобы объединились полные датафреймы - надо убрать окончание "_small" в ячейке ниже. Далее будет использовано название train. После объединения полных датафреймов это название менять не надо.

In [5]:
train = train_4_col_3_small.merge(train_4_col_2_small, on='id', how='left').merge(train_4_col_small, on='id', how='left')
train.head(2).T

,0,1
id,3edeb34cf93f490ff760af85,c98febd50dad3cc0ffc86085
fields,"[{'field': {'name': 'Женский гардероб', 'id': ...","[{'field': {'name': 'Детский гардероб', 'id': ..."
category,9,22
subcategory,914,2202
name,Сумка DG,Комплект
description,NaN,8-12 лет
price,199900,35000


In [6]:
# Функция, которая извлекает значений ключей из поля "fields"
def valextractor(field, key1=0, key2='name'):
    value = eval(field[1:-1])[key1]['field'][key2] 
    return value

In [7]:
# проверяем функцию. работает.
field = train.iloc[0,1]
valextractor(field, key1=0,key2='name')

'Женский гардероб'

In [8]:
# функция, которая доавляет новые категории
def getcat(df):
    df_ = df.copy()
    df_['fd_name1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='name')))
    df_['fd_name2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='name')))
    df_['fd_id1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='id')))
    df_['fd_id2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='id')))
    df_['fd_slug1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='slug')))
    df_['fd_slug2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='slug')))
    df_['fd_slug_id1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='slug_id')))
    df_['fd_slug_id2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='slug_id')))
    return df_

In [9]:
# применим функцию
train = train.pipe(getcat)

In [10]:
# удалим задублированные столбцы
train.drop(columns=['fd_id1','fd_id2','fields' ],axis=1, inplace=True)
# посмотрим что получилось
train.head(50).T

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
id,3edeb34cf93f490ff760af85,c98febd50dad3cc0ffc86085,ade01e13912a46a99134cc75,ab3e6941c11304c1519aef75,252452a91c944a22c276d995,364f7659a4e9ba6c53e35495,c3d87065e4dbc080f400bb85,22e6a518d53f490f6ce7fd85,c6bd550b7f1304c1f7fda485,117b885a3cf9554042ae2285,...,3438949c05e3b470c984ea95,266fc4d1395ccdeb3428fe85,62c792b5c1141fee419acc95,3aec3024bc0e788cdb046795,62df3b3861f95540e9834385,3a80d58fd82d9e58ce4aed85,30b215c9295ccdeb1abe2195,060658c789d3f35d14320d75,4a23d59792944a225801d395,3105314ef46c1e2633b58195
category,9,22,22,22,22,9,15,4,3,2,...,3,3,22,14,16,22,3,9,22,100
subcategory,914,2202,2204,2209,2204,902,1508,403,301,208,...,317,314,2209,1403,1608,2209,314,902,2211,10001
name,Сумка DG,Комплект,Пальтишко демисезонное,Attipas,Жилет теплый,"Босоножки, сабо",Жесткий диск для ноутбука Seagate 320gb,Ковер 324 на 240,Кенгуру,Машинка автомат,...,Электронная качель,Покебол со случайной минифигуркой покемона внутри,Ботинки зимние,Духи Lancome la vie est belle,Набор для маникюра,Босоножки для танцев,Коврик музыкальный,Туфли р .36,Нарядное платье,Детское питание
description,NaN,8-12 лет,"На девочку 1,5 г,состояние хорошее","Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",NaN,"Новые, текстильные босоножки, 37 размер в разм...",Seagate Модель St320lt020 HDD для ноутбуков ...,"Состояние отличное,без дефектов","Продам кенгуру 350 , ванночка с горкой 250",NaN,...,"Электронная качель Baby Care Riva, удобная, ко...",Размер покебола: 7 см.,"Зимние, размер 20-21,( так написано на подошве...","Edp! Оригинал! Объём 30 мл, осталось половина....",На моем профиле моного интересных вещей,Для танцев стелька 22см,"Для возраста 3+, несколько режимов работы","Туфли, кожа натуральная. Подходят для школы. О...","Красивое нарядное платье, размер 104, внутри т...",Продам 2 пачки новых каш с грушей и бананом. Ц...
price,199900,35000,30000,80000,50000,30000,150000,200000,35000,350000,...,260000,30000,80000,90000,25000,50000,30000,100000,40000,10000
fd_name1,Женский гардероб,Детский гардероб,Детский гардероб,Детский гардероб,Детский гардероб,Женский гардероб,Компьютерная техника,Для дома и дачи,Детские товары,Бытовая техника,...,Детские товары,Детские товары,Детский гардероб,Красота и здоровье,Услуги,Детский гардероб,Детские товары,Женский гардероб,Детский гардероб,Прочее
fd_name2,Аксессуары,Аксессуары,Верхняя одежда,Обувь,Верхняя одежда,Обувь,Комплектующие и запчасти,Текстиль и ковры,Другое,Стиральные машины,...,Обустройство детской,Игрушки и игры,Обувь,Парфюмерия,Красота и здоровье,Обувь,Игрушки и игры,Обувь,Платья и юбки,Другое
fd_slug1,zhenskaya-odezhda,detskaya-odezhda,detskaya-odezhda,detskaya-odezhda,detskaya-odezhda,zhenskaya-odezhda,kompyutery,dom-dacha,detskie,bytovaya-tekhnika,...,detskie,detskie,detskaya-odezhda,krasota-i-zdorove,uslugi,detskaya-odezhda,detskie,zhenskaya-odezhda,detskaya-odezhda,prochee
fd_slug2,aksessuary,aksessuary,verhnyaya-odezhda,obuv,verhnyaya-odezhda,obuv,komplektuyushchie,tekstil-kovry,drugoe,stiralnye-mashiny,...,mebel,kukly-igrushki,obuv,parfyumeriya,krasota-zdorove,obuv,kukly-igrushki,obuv,platya-i-yubki,drugoe


In [11]:
train.to_csv('data/train.csv')

### Проведем морфологический анализ

In [12]:
import re
import pymorphy2 # Морфологический анализатор.
from collections import Counter 

In [13]:
morph=pymorphy2.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь. теперь переменная morph может проводить морфологический анализ

In [14]:
def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)  # Хочу получить все значимые слова, но в уже нормальной форме
                break
    return words

In [15]:
pd.set_option('max_rows',100)

In [16]:
%%time
train['morph_description'] = train['description'].fillna('').astype('object')\
.map(lambda text: ' '.join([morph.parse(r)[0].normal_form for r in re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)]))


Wall time: 876 ms


In [17]:
train.loc[3,'morph_description']

'размер прочный мягкий не скользящий обувь для маленькая гибкий подошва обеспечивать много место для движение пальчик обувь производиться без использование клеить и токсичный вещество благодаря небольшой отверстие в подошва позволять дышать ножка машинный стирка быстро сохнуть состав материал подошва термопластичный силикон материал носочек хлопок спаднекс резиновый нить полиуритановое нить обувь помогать правильный моторный развитие'

In [18]:
%%time
train['morph_description'] = train['description'].fillna('').astype('object')\
.map(lambda text: ' '.join(getMeaningfullWords(text)))

Wall time: 826 ms


In [19]:
train.loc[3,'morph_description']

'размер прочный мягкий скользящий обувь маленькая гибкий подошва обеспечивать место движение пальчик обувь производиться использование клей исполняющий токсичный вещество небольшой отверстие век подошва позволять ножка машинный стирка сохнуть состав материал подошва термопластичный силикон материал носочек хлопок спаднекс резиновый нить полиуритановое нить обувь помогать правильный моторный развитие'

#### По времени оба варианта примерно одинаковы. посмотрим еще на объемах чуть побольше

## Получим списки с частотами по Tf-Idf

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [21]:
corpus = train['morph_description'] # Создадим корпус значимых слов
n = 3
example = train.loc[n,'morph_description'] # получаем n-ю строку

In [22]:
tfCounter=TfidfVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')
analyze = tfCounter.build_analyzer()
res=tfCounter.fit_transform(corpus) # Скармливаем сюда корпус значимых слов

In [23]:
res2=analyze(example) # подаем ранее пролученную n-ю строку

# Получим значения частот всех слов n-й строки
tfs=list(set(res[n][0, tfCounter.vocabulary_.get(k)] for k in res2 if k in tfCounter.vocabulary_.keys())) # укажем номер n 

# получим значения частот только тех слов, значения которых превышают среднее значение частот n-й строки
tfs2=[k for k in tfs if k>np.average(tfs)]

# или получим значения частот только тех слов, значения которых превышают среднее значение + Стандартное отклонение 
# частот n-й строки
#tfs2=[k for k in tfs if k>np.average(tfs)+np.std(tfs)]

print({w:res[n][0, tfCounter.vocabulary_[w]] for \
          w in res2 if w in tfCounter.vocabulary_.keys() and\
       res[n][0, tfCounter.vocabulary_[w]] in tfs2})


{'обувь': 0.29678410230734736, 'подошва': 0.27233545338034, 'материал': 0.19785606820489826, 'нить': 0.19785606820489826}


Или  вариант со средним значением

In [24]:
n = 3
example = train.loc[n,'morph_description'] # получаем строку

res2=analyze(example) # подаем строку


tfs=list(set(res[n][0, tfCounter.vocabulary_.get(k)] for k in res2 if k in tfCounter.vocabulary_.keys()))

#tfs2=[k for k in tfs if k>np.average(tfs)]
tfs2=[k for k in tfs if k>np.average(tfs)+np.std(tfs)]

print({w:res[n][0, tfCounter.vocabulary_[w]] for \
          w in res2 if w in tfCounter.vocabulary_.keys() and\
       res[n][0, tfCounter.vocabulary_[w]] in tfs2})


{'обувь': 0.29678410230734736, 'подошва': 0.27233545338034}


Возможно это еще пригодится. Вставить это в новый столбец, старое описание удалить, запустить dictVectorizer - , должно получиться покомпактнее, возможно будет и результат получше.

In [25]:
# Просто почитать описания
row = 31
train.loc[row,'description']

'Одевала один раз.покупала за 2000.с теплым подкладом.'

In [26]:
# Просто посмотреть список значимых слов
train['morph_description']

0                                                      
1                                                   год
2                         девочка год состояние хороший
3     размер прочный мягкий скользящий обувь маленьк...
4                                                      
5     новое текстильный босоножка размер век размер ...
6     модель ноутбук ёмкость накопитель гб отличный ...
7                             состояние отличный дефект
8                продать кенгуру ванночка секунда горка
9                                                      
10    люлька возраст люлька век машина задний сидени...
11    новое платье посол чистка одевать весь один ра...
12                         размер рост чистенький пятно
13    состояние хороший весь целое прошлый осень над...
14    отдать манеж исполняющий куча детский вещий бо...
15    ботинок иза натуральный кожа мягкий благородны...
16    новое век коробка секунда бирка продаба скидка...
17                                            це

### Поробуем Tf-Idf в "чистом" виде

In [27]:
# пока задаим X таким
X = train['morph_description']

In [28]:
# Задаим стоп слова
stop_words = ['век','год','месяц','день','час','минута','секунда','иза','продать','купить','звонить','цена','влюбиться',
'состояние','дорг','размер','танк','продавать','сантиметр', 'брать','быть','пользование','понадобиться','любой',
'рубль','контакт','продаваться']
print('Количество стоп-слов составляет:',len(stop_words))

Количество стоп-слов составляет: 27


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=stop_words)
Tfidf_array = vectorizer.fit_transform(X)
#print('Список слов, определяющих частотный список:',vectorizer.get_feature_names())
print('Размеры (shape) полученного массива Tfidf_array (строк, слов):', Tfidf_array.shape)

Размеры (shape) полученного массива Tfidf_array (строк, слов): (100, 427)


In [30]:
# Так можно получить матрицу частот - пока на всякий случай..
#pd.set_option('display.max_rows', 500) 
#df_tfidf = pd.DataFrame(Tfidf_array.toarray())

#### Далее идут наработки по созданию числовых лэйблов и One Hote Encoding из них. Хотя пока еще рановато. Надо еще tf-idf сделать

In [31]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

In [32]:
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import xgboost as xgb


 Зададим селектор

In [33]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X_, y=None): # 
        return self
    def transform(self, X_):
        return X_.loc[:, self.attribute_names].values # 

Зададим списки столбцов

In [34]:
# получим список столбцов 
full_columns_list = list(train) 

# список числовых категориальных столбцов
num_cat_columns = full_columns_list[1:3] # ['category', 'subcategory']

# список буквенных категориальных столбцов
let_cat_columns = full_columns_list[6:12]  # ['fd_name1', 'fd_name2', 'fd_slug1', 'fd_slug2', 'fd_slug_id1', 'fd_slug_id2'] 

tfidf_columns = ['morph_description', 'name']

# целвое значение
target = ['price']
print('num_cat_columns :',num_cat_columns)
print('let_cat_columns :',let_cat_columns)
print('tfidf_columns   :',tfidf_columns)

other = list(set(full_columns_list) - set(num_cat_columns) - set(let_cat_columns) - set(target))
print('Остаток         :', other)

num_cat_columns : ['category', 'subcategory']
let_cat_columns : ['fd_name1', 'fd_name2', 'fd_slug1', 'fd_slug2', 'fd_slug_id1', 'fd_slug_id2']
tfidf_columns   : ['morph_description', 'name']
Остаток         : ['morph_description', 'description', 'name', 'id']


In [35]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

Зададим 3 папйплайна для последующей сборки

ТУТ где то уже пошли косяки, где-то что-то не в том виде наверное подал.

In [ ]:
num_pipeline = Pipeline([
('selector_1',   ColumnSelector(let_cat_columns)),# определим категориальные словестные 
                                            #['fd_name1', 'fd_name2', 'fd_slug1', 'fd_slug2', 'fd_slug_id1', 'fd_slug_id2']
('lbl_enc', LabelBinarizer())    
#('scaler',  StandardScaler()), # 
])

In [ ]:
cat_pipeline = Pipeline([  
('selector_2', ColumnSelector(num_cat_columns)), # определим категориальные числовые столбцы: ['category', 'subcategory']
('OHE', OneHotEncoder())
])

In [ ]:
tfidf_pipeline = Pipeline([  
('selector_3', ColumnSelector(tfidf_columns)), # определим столбцы для  tfidf  преобразования: ['morph_description', 'name']
('Tf*Idf', TfidfVectorizer(stop_words=stop_words))
])

Соберем в один

In [ ]:
featureunion_pipeline = FeatureUnion(transformer_list=[
("pipe_1", num_pipeline),
("pipe_2", cat_pipeline),
("pipe_3", tfidf_pipeline),    
])

Подаим его на вход предсказателя

In [ ]:
xgb_model = xgb.XGBClassifier()

pipe= Pipeline([('f', featureunion_pipeline), ('x', xgb_model)])

In [ ]:
pipe.fit(train)

In [ ]:
# Еще есть такой кариант сборки пайплайнов
col_transform = ColumnTransformer([
    ('OHE', OneHotEncoder(), [204]), # 
    ('scaler',StandardScaler(), slice(0,204))
     ])

full_pipeline_4 = Pipeline([
('ct', ct),
('clf', LogisticRegression())
])